In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2
plt.style.use('ggplot')
import sys 
sys.path.append("/scratch/izar/kapps/DEX-Cyclic-Arbitrage/")
from config.get import cfg
# from tensorflow import keras
# from tensorflow.keras import layers
import autoencoders

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# loading data

In [8]:
X_train = np.load(cfg['files']["raw_train_features"])
X_test  = np.load(cfg['files']["raw_test_features"])

# loading model

In [9]:

model_name , autoencoder = autoencoders.linear_with_reshape()
autoencoder.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3, 600, 2)]       0         
                                                                 
 reshape_2 (Reshape)         (None, 3600)              0         
                                                                 
 dense_2 (Dense)             (None, 100)               360100    
                                                                 
 dense_3 (Dense)             (None, 3600)              363600    
                                                                 
 reshape_3 (Reshape)         (None, 3, 600, 2)         0         
                                                                 
Total params: 723,700
Trainable params: 723,700
Non-trainable params: 0
_________________________________________________________________


# Training

/!\ does not run yet (talos code reorganisation)

In [6]:
# fit function for talos
def fit_model(X_train, y_train, X_val, y_val, params):
    # build model
    # fit model
    _,autoencoder = autoencoders.Complex_CNN    
    
    out = autoencoder.fit(
        X_train, X_train,
        shuffle=True,
        verbose=1,
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        validation_data=[x_val, y_val],
        callbacks=[talos.utils.early_stopper(params['epochs'])],
    )
    plt.plot(out.history["loss"])
    plt.show()
    return out, autoencoder

In [ ]:
fit_model(autoencoder,X_train, y_train, X_val, y_val, params)

## hyperparameters tuning

In [ ]:
from tensorflow.keras.activations import relu, elu
## the parameter that you want to be optimized are defined in this dictionnary
p = {
    'activation':['relu', 'elu'],
    'losses': ['logcosh'],
    'shapes': ['brick'],
    'first_neuron': [32],
    'dropout': [.2, .3],
    'batch_size': [32, 64],
    'epochs': [1]
}

In [ ]:
scan_object = talos.Scan(
    x=tX_train, y=y_train, params=p, 
    model=fit_model, 
    experiment_name='ml4f-autoencoder', 
    x_val=X_val, y_val=y_val
) 

In [ ]:
scan_object.evaluate_models(
    x_val=x_val,
    y_val=y_val,
    n_models=10,
    metric='f1score',
    folds=5,
    shuffle=True,
    average='binary',
    asc=False
)

In [ ]:
scan_object.data

In [ ]:
autoencoder.save('autoencoder_model')

# Load and extract encoder

In [5]:
cfg["models"]["autoencoder"]

'/scratch/izar/kapps/DEX-Cyclic-Arbitrage/models/embedding/autoencoder/'

# Model  evaluation

In [19]:
tX_train = np.load(cfg['files']['raw_train_features'])
tX_test  = np.load(cfg['files']['raw_test_features'])
N_train, N_test = tX_train.shape[0], tX_test.shape[0]
N = N_train + N_test

In [53]:
#from sklearn.metrics import mean_squared_error

def mean_squared_error(x,y):
    return np.sum((x-y)**2)/x.size

In [54]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=100)
ttX_train = tX_train.reshape(N_train,-1)
ttX_test = tX_test.reshape(N_test,-1)
pca.fit(ttX_train)
pca_train = mean_squared_error(ttX_train, pca.inverse_transform(pca.transform(ttX_train)))
pca_test =  mean_squared_error(ttX_test, pca.inverse_transform(pca.transform(ttX_test)))
print(f"MSE train : {pca_train} \nMSE test  : {pca_test}")

MSE train : 2701.1471313988195 
MSE test  : 2875.2690935465525


In [64]:
perf = pd.DataFrame({ 'MSE': [35931.47411685519, 34284.88200871408, pca_train, pca_test],'model': ['Autoencoder','Autoencoder', 'PCA','PCA'],'set': ['train set','test set','train set','test set']})

In [73]:
import plotly.express as px
fig = px.bar(perf, x="set", y="MSE",
             color='model', barmode='group',
             height=400, range_y=(2000,40000))
fig.update_layout(title_text="Embedding performance : Autoencoder vs PCA")
fig.update_xaxes(title="")
fig.write_html(f"{cfg['fig_dir']['embedding']}performance_autoencoder.html",full_html=False, include_plotlyjs=False)
fig.show()